In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data
Load Text novel Sherloch Holmes dalam bahasa indonesia

In [2]:
# open text file and read in data as 'text'
with open('data/Sherlockholmesindo.txt', 'r') as f:
    text = f.read()

Let's check the first 100 characters, make sure everything is ok.

In [4]:
text[:200]

'PENELUSURAN\nBENANG MERAH\n\n\n\nSir Arthur Conan Doyle – Penelusuran Benang Merah\n\nBAGIAN I\nSALINAN CATATAN HARIAN\nDOKTER JOHN H. WATSON, PENSIUNAN\nDEPARTEMEN MEDIS ANGKATAN DARAT\n\n\n\n1\n\nSir Arthur Conan D'

## Tokenization
creating couple __dictionaries__ to convert the characters to and from integers.<br>
Encoding the characters as integers makes it easier to use as input in the network

In [13]:
# encoding the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

Now see the text encoded as integers

In [18]:
encoded = np.array([char2int[ch] for ch in text])
encoded[:200]

array([62, 12, 35, 12, 39, 55, 23, 55,  9, 80, 35, 11, 70, 12, 35, 80, 35,
       69, 20, 41, 12,  9, 80, 36, 11, 11, 11, 11, 23, 59,  6, 20, 80,  6,
        4, 52, 63,  6, 20, 50, 29, 58, 82, 58, 20,  1, 29, 42, 30, 25, 20,
       53, 20, 62, 25, 58, 25, 30, 63, 76, 63,  6, 82, 58, 20, 70, 25, 58,
       82, 58, 61, 20, 41, 25,  6, 82, 52, 11, 11, 70, 80, 69, 51, 80, 35,
       20, 51, 11, 23, 80, 39, 51, 35, 80, 35, 20, 50, 80, 47, 80, 47, 80,
       35, 20, 36, 80,  9, 51, 80, 35, 11,  1, 66, 72, 47, 12,  9, 20, 74,
       66, 36, 35, 20, 36, 17, 20, 44, 80, 47, 23, 66, 35, 46, 20, 62, 12,
       35, 23, 51, 55, 35, 80, 35, 11,  1, 12, 62, 80,  9, 47, 12, 41, 12,
       35, 20, 41, 12,  1, 51, 23, 20, 80, 35, 69, 72, 80, 47, 80, 35, 20,
        1, 80,  9, 80, 47, 11, 11, 11, 11, 34, 11, 11, 23, 59,  6, 20, 80,
        6,  4, 52, 63,  6, 20, 50, 29, 58, 82, 58, 20,  1])

## Pre-processing the data
As you can see in our char-RNN image above, our LSTM expects an input that is one-hot encoded meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!

In [20]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elemnets with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [21]:
# Check if the function is works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches
To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps.

In [37]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    # Get the number of batches we can make
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

## Test the Implementation

In [39]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [40]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[62 12 35 12 39 55 23 55  9 80]
 [61 59 11  6 25 40 82 58 16  6]
 [63 52 20 40 25  0 82  0 31 63]
 [27 82  6 82 27 82 20 59 58 59]
 [58 59 20 68 59 82  8 48 11 48]
 [25  0 64 63 58 63 52 20 76 82]
 [ 4 20 68 59 20  0 25 13 82 20]
 [68 82 20 68 59 20 31 25  6  4]]

y
 [[12 35 12 39 55 23 55  9 80 35]
 [59 11  6 25 40 82 58 16  6 25]
 [52 20 40 25  0 82  0 31 63 82]
 [82  6 82 27 82 20 59 58 59 20]
 [59 20 68 59 82  8 48 11 48 23]
 [ 0 64 63 58 63 52 20 76 82  4]
 [20 68 59 20  0 25 13 82 20 68]
 [82 20 68 59 20 31 25  6  4 82]]


## Defining the network with Pytorch

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

In [41]:
# check if GPU is available

train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU !')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU !


In [43]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                    drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr 
        
        # Creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define layers of the model
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                           dropout=drop_prob, batch_first=True)
        
        ## define Dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        """ Forward pass through the network.
        These inputs are x, and the hidden/cell state 'hidden'."""
        
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        # Pass through a dropout layer 
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## Put the x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden
        # Initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                     weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                     weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        return hidden
    

## Time to train
The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [47]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, 
          val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if (train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # Initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode out data and make them torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # Zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # 'clip_grad_norm' helps prevent the exploding gradient problems in RNNs/ LSTMs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode out data and make them torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                    val_losses.append(val_loss.items())
                
                net.train() # reset to train mode after iterating through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                     "Step: {}...".format(conter),
                     "Loss: {:.4f}...".format(loss.item()),
                     "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model
Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

### Set model hyperparameters
#### Set and print the net

In [58]:
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(84, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=84, bias=True)
)


### Set training hyperparameters

In [59]:
batch_size = 128
seq_length = 100
n_epochs = 20

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size,
     seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/20... Step: 10... Loss: 3.2017... Val Loss: 3.1341
Epoch: 1/20... Step: 20... Loss: 3.0981... Val Loss: 3.0496
Epoch: 1/20... Step: 30... Loss: 3.0771... Val Loss: 3.0343
Epoch: 1/20... Step: 40... Loss: 3.0634... Val Loss: 3.0248
Epoch: 1/20... Step: 50... Loss: 3.0362... Val Loss: 3.0240
Epoch: 1/20... Step: 60... Loss: 3.0596... Val Loss: 3.0189
Epoch: 1/20... Step: 70... Loss: 3.0587... Val Loss: 3.0119
Epoch: 1/20... Step: 80... Loss: 3.0176... Val Loss: 2.9938
Epoch: 2/20... Step: 90... Loss: 3.0002... Val Loss: 2.9580
Epoch: 2/20... Step: 100... Loss: 2.8801... Val Loss: 2.8323
Epoch: 2/20... Step: 110... Loss: 2.7126... Val Loss: 2.6608
Epoch: 2/20... Step: 120... Loss: 2.5966... Val Loss: 2.5453
Epoch: 2/20... Step: 130... Loss: 2.5304... Val Loss: 2.4601
Epoch: 2/20... Step: 140... Loss: 2.4307... Val Loss: 2.3886
Epoch: 2/20... Step: 150... Loss: 2.3736... Val Loss: 2.3237
Epoch: 2/20... Step: 160... Loss: 2.3422... Val Loss: 2.2805
Epoch: 3/20... Step: 170... Loss:

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [61]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens':net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [70]:
def predict(net, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if(train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if (train_on_gpu):
        p = p.cpu() # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())

    # retirn the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

## Priming and generating text

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [72]:
def sample(net, size, prime='Sebuah', top_k=None):
    
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)
    
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, char[-1], h, top_k=top_k)
        chars.append(char)
    
    return ''.join(chars)

In [73]:
print(sample(net, 1000, prime='Sebuah', top_k=5))

Sebuah
sekali. Keliar kepada satu sebagai atas. Kepalanya memperhitapikannya di rumah dan tersangka.
"Saat berhasil menghuni kepada Sir Charles sambil merupakan siapa darepan yang mengirim kepada perasaan yang telah berusaha menemui kosis, sisa, dari sekarang kita menceritakan kami. Dan kau
membungkik satu sesuatu saya, masih bisa
membunyukan sebuah bahwa dia melirik dengan mengetahui suara pertamanya. Sir Henry tampaknya
tidak tuk penasir. Dengan kereta sesuatu yang tampaknya saya sendiri
dari banyak pelayan di tungga. Derek penuh mereka.
"Kualas ke tahan kami meminta kasus ini. Tapi tunya-aku
pasti sama sekali tiba di sekitar kemerihaan sementara
kepercayaan yang berar dan membantu, darah semuanya yang meraka murak di kejahatan, dan bagiku tanpa sebelum itu senapannya, kemarin kuas. Kelemparnya kami sambil semua ini berhasil berada di
depan sesuatu yang mereka. Kemudian dia berhasil menghadapi kering dereman. Saya
selalu selelari menenjikkan semua kumis dan menunjukkan kereta
dengan 

## Evaluation

epoch 20 is still not good enough, need to retrain

### Train in more epochs (100) epochs

In [74]:
batch_size = 128
seq_length = 100
n_epochs = 100

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size,
     seq_length=seq_length, lr=0.001, print_every=20)


Epoch: 1/100... Step: 20... Loss: 1.2392... Val Loss: 1.3125
Epoch: 1/100... Step: 40... Loss: 1.1963... Val Loss: 1.3145
Epoch: 1/100... Step: 60... Loss: 1.2326... Val Loss: 1.3055
Epoch: 1/100... Step: 80... Loss: 1.2058... Val Loss: 1.3046
Epoch: 2/100... Step: 100... Loss: 1.1981... Val Loss: 1.3023
Epoch: 2/100... Step: 120... Loss: 1.2297... Val Loss: 1.3066
Epoch: 2/100... Step: 140... Loss: 1.1863... Val Loss: 1.2984
Epoch: 2/100... Step: 160... Loss: 1.2309... Val Loss: 1.2984
Epoch: 3/100... Step: 180... Loss: 1.2047... Val Loss: 1.2950
Epoch: 3/100... Step: 200... Loss: 1.1883... Val Loss: 1.2972
Epoch: 3/100... Step: 220... Loss: 1.1798... Val Loss: 1.2918
Epoch: 3/100... Step: 240... Loss: 1.1659... Val Loss: 1.2913
Epoch: 4/100... Step: 260... Loss: 1.1723... Val Loss: 1.2881
Epoch: 4/100... Step: 280... Loss: 1.1493... Val Loss: 1.2929
Epoch: 4/100... Step: 300... Loss: 1.1725... Val Loss: 1.2915
Epoch: 4/100... Step: 320... Loss: 1.1949... Val Loss: 1.2876
Epoch: 5/100

In [75]:
# change the name, for saving multiple files
model_name = 'rnn_100_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens':net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [76]:
print(sample(net, 1000, prime='Sebuah', top_k=5))

Sebuah petenakan. Aku sudah memenuhi
mana asalnya, bagui biasanya tidak bisa membuktikan keluarga Anda
di Baskerville Hall."
"Aku harus pergi ke London, Strant itu bagaimana tulisan di sini?"
"Aku tidak mengusapnya semakin mantap."
"Sebaiknya kau tetap mengindah saja sampai akhirnya kau sedang
bertindak lagi, Sir. Dia sempat.
"Dolim tiga buni penjelasan yang luar biasa telah menyusun tampak lain yang berkeliaran di rawa-rawa daripada kembali. Sekarang ia mungkin sudah
mengambil situasinya tanpa satu hal, dan
tanpa pembunuhan sebelumnya. Dua kalinya akan
berani berlalu ke jendela dan menutup molit di dalam sakunya. Itu suara penghuninya, yang mengungkapkannya dengan begitu jutulan, sebagaimana kusangkinya. Apa
yang diketahuinya?"
"Bagaimana dengan kelompok ini di sini."


113

Sir Arthur Conan Doyle – Penelusuran Benang Merah

"Siapa penghuninya?" tanya
Mr. Barker
membuka paling bagai
berakhir. "Tentu saja, ia salah satu dua bukan
benar dia tidak berani menyerahkan kepergian seperti pun